### 1차 코드

In [ ]:
from ultralytics import YOLO
import pyrealsense2 as rs
import numpy as np
import cv2
import math
import time

# -------------------------------------------------------
# 0) 설정
# -------------------------------------------------------
MODEL_PATH = "/home/dw/ws_job_msislab/amr_project/src/runs/obb/smoke_test_v1/weights/best.pt"
CONF_TH = 0.25
IMGSZ = 640
DEVICE = 0  # GPU 없으면 "cpu"

COLOR_W, COLOR_H, FPS = 640, 480, 30
DEPTH_W, DEPTH_H = 640, 480

# depth 샘플링: 중심 주변에서 N x N 픽셀을 뽑아 median
DEPTH_PATCH = 9          # 홀수 권장 (예: 5, 7, 9, 11)
DEPTH_MIN_M = 0.10       # 유효 depth 최소
DEPTH_MAX_M = 2.50       # 유효 depth 최대`

# -------------------------------------------------------
# 1) YOLO 로드
# -------------------------------------------------------
model = YOLO(MODEL_PATH)

# -------------------------------------------------------
# 2) RealSense 파이프라인
# -------------------------------------------------------
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, COLOR_W, COLOR_H, rs.format.bgr8, FPS)
config.enable_stream(rs.stream.depth, DEPTH_W, DEPTH_H, rs.format.z16, FPS)

profile = pipeline.start(config)

# color에 depth 정렬
align = rs.align(rs.stream.color)

# intrinsics (color stream 기준)
color_stream = profile.get_stream(rs.stream.color).as_video_stream_profile()
intr = color_stream.get_intrinsics()
fx, fy, ppx, ppy = intr.fx, intr.fy, intr.ppx, intr.ppy

print("[INFO] RealSense started")
print(f"[INFO] intrinsics: fx={fx:.2f}, fy={fy:.2f}, ppx={ppx:.2f}, ppy={ppy:.2f}")
print("[INFO] Press 'q' to quit")

def median_depth_in_patch(depth_frame, u, v, patch=9):
    """
    (u,v) 중심 주변 patch x patch 영역의 depth(m) median 반환.
    depth_frame은 align된 depth_frame이어야 함.
    """
    half = patch // 2
    h, w = depth_frame.get_height(), depth_frame.get_width()

    u0 = max(0, u - half)
    u1 = min(w - 1, u + half)
    v0 = max(0, v - half)
    v1 = min(h - 1, v + half)

    vals = []
    for yy in range(v0, v1 + 1):
        for xx in range(u0, u1 + 1):
            d = depth_frame.get_distance(xx, yy)  # meters
            if DEPTH_MIN_M <= d <= DEPTH_MAX_M:
                vals.append(d)

    if not vals:
        return None
    return float(np.median(vals))

def compute_obb_axes_and_angle(pts):
    """
    pts: (4,2) float32, OBB 4점(시계방향)
    return:
      cx,cy (float), v_long(unit), v_short(unit),
      len_long(px), len_short(px),
      angle_deg (vertical 기준 0°, 오른쪽 +, 왼쪽 -)
    """
    cx, cy = pts.mean(axis=0)

    p0, p1, p2, p3 = pts

    e01 = p1 - p0
    e12 = p2 - p1

    len01 = np.linalg.norm(e01)
    len12 = np.linalg.norm(e12)
    if len01 < 1e-6 or len12 < 1e-6:
        return None

    # 더 긴 변을 장축으로 선택 (너 케이스 70도 튐 해결)
    if len01 >= len12:
        v_long = e01
        v_short = e12
        len_long, len_short = len01, len12
    else:
        v_long = e12
        v_short = e01
        len_long, len_short = len12, len01

    v_long = v_long / (len_long + 1e-12)
    v_short = v_short / (len_short + 1e-12)

    # 장축 방향 통일: 항상 위쪽(-y) 향하도록 (부호 안정화)
    if v_long[1] > 0:
        v_long = -v_long

    # 세로 기준 각도:
    # 0° = 위쪽(-y)과 일치
    # 오른쪽 기울면 +, 왼쪽 기울면 -
    angle_deg = math.degrees(math.atan2(v_long[0], -v_long[1]))

    return cx, cy, v_long, v_short, float(len_long), float(len_short), float(angle_deg)

try:
    while True:
        frames = pipeline.wait_for_frames()
        frames = align.process(frames)

        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue

        color = np.asanyarray(color_frame.get_data())

        # ---------------------------
        # YOLO OBB inference
        # ---------------------------
        results = model.predict(
            source=color,
            imgsz=IMGSZ,
            conf=CONF_TH,
            device=DEVICE,
            verbose=False
        )
        r = results[0]

        if r.obb is not None and len(r.obb) > 0:
            boxes = r.obb.xyxyxyxy.cpu().numpy()  # (N,4,2)
            confs = r.obb.conf.cpu().numpy()

            # 가장 높은 conf 1개만 선택(테스트용)
            best_i = int(np.argmax(confs))
            box = boxes[best_i]
            conf = float(confs[best_i])

            pts_i = box.astype(np.int32)
            pts = box.astype(np.float32)

            # OBB 외곽선
            cv2.polylines(color, [pts_i], True, (255, 0, 0), 2)

            axes = compute_obb_axes_and_angle(pts)
            if axes is not None:
                cx, cy, v_long, v_short, len_long, len_short, angle_deg = axes
                cx_i, cy_i = int(cx), int(cy)

                # 박스 전체 십자가
                long_p1 = (int(cx - v_long[0]*len_long/2), int(cy - v_long[1]*len_long/2))
                long_p2 = (int(cx + v_long[0]*len_long/2), int(cy + v_long[1]*len_long/2))
                short_p1 = (int(cx - v_short[0]*len_short/2), int(cy - v_short[1]*len_short/2))
                short_p2 = (int(cx + v_short[0]*len_short/2), int(cy + v_short[1]*len_short/2))
                cv2.line(color, long_p1, long_p2, (0, 255, 255), 2)
                cv2.line(color, short_p1, short_p2, (0, 255, 255), 2)

                # depth (중심 주변 median)
                Z = median_depth_in_patch(depth_frame, cx_i, cy_i, patch=DEPTH_PATCH)

                if Z is not None:
                    # deproject: pixel + depth -> 3D (camera frame)
                    X, Y, Zm = rs.rs2_deproject_pixel_to_point(intr, [cx_i, cy_i], Z)
                    # rs2_deproject_pixel_to_point returns [X,Y,Z] in meters
                    dist = math.sqrt(X*X + Y*Y + Zm*Zm)

                    text1 = f"conf={conf:.2f}  angle={angle_deg:+.1f} deg"
                    text2 = f"cam XYZ=({X:+.3f},{Y:+.3f},{Zm:+.3f}) m  dist={dist:.3f} m"
                else:
                    text1 = f"conf={conf:.2f}  angle={angle_deg:+.1f} deg"
                    text2 = "depth: N/A (try adjust DEPTH_PATCH / object surface)"

                cv2.putText(color, text1, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(color, text2, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

        cv2.imshow("YOLO11-OBB + Depth (Camera frame)", color)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

finally:
    pipeline.stop()
    cv2.destroyAllWindows()
    print("[INFO] stopped")


[INFO] RealSense started
[INFO] intrinsics: fx=392.57, fy=392.14, ppx=319.22, ppy=241.26
[INFO] Press 'q' to quit
[INFO] stopped
